In [ ]:
# Yottixel's patching algorithm + our pipeline

In [17]:
# (A) System package for OpenSlide tools
!apt-get update -qq && \
 apt-get install -y -qq openslide-tools

# (B) Python packages via pip
#  • Use built-in TensorFlow
#  • Install OpenSlide Python bindings
#  • Install bitarray
#  • Install bitarray-hardbyte from its GitHub source
!pip install --upgrade pip
!pip install openslide-python tensorflow

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
# gigapath encoder

import os
from pathlib import Path
import torch
import timm
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import subprocess

# insert huggingface token
os.environ['HF_TOKEN'] = '' # gitignore
hf_token = os.environ['HF_TOKEN']
assert hf_token, 'HF_TOKEN is not set'

# google cloud authentication
from google.colab import auth
from google.cloud import storage

auth.authenticate_user()
BUCKET_NAME = 'bracs-dataset-bucket'

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

# download gigapath encoder
import timm

tile_encoder = timm.create_model("hf_hub:prov-gigapath/prov-gigapath", pretrained=True).cuda().eval()
print("Tile Encoder loaded.")
print("Total parameters:", sum(p.numel() for p in tile_encoder.parameters()))

# Image transform
transform = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])





config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Tile Encoder loaded.
Total parameters: 1134953984


In [11]:
# lists slides in a split ['train', 'test', 'val']
def list_slides(split):
    prefixes = [f"BRACS/BRACS_WSI/{split}/Group_AT/",
                f"BRACS/BRACS_WSI/{split}/Group_BT/",
                f"BRACS/BRACS_WSI/{split}/Group_MT/"]
    slides = []
    for pref in prefixes:
        blobs = client.list_blobs(bucket.name, prefix=pref)
        for b in blobs:
            if b.name.lower().endswith(".svs"):
                slides.append(b.name)
    return slides

split = 'train'

len(list_slides(split))

395

In [32]:
import openslide, numpy as np
import cv2
from cv2 import filter2D

%matplotlib inline
import matplotlib.pyplot as plt

# https://github.com/FarhadZanjani/Histopathology-Stain-Color-Normalization/blob/master/ops.py
def RGB2HSD(X):
    eps = np.finfo(float).eps
    X[np.where(X==0.0)] = eps

    OD = -np.log(X / 1.0)
    D  = np.mean(OD,3)
    D[np.where(D==0.0)] = eps

    cx = OD[:,:,:,0] / (D) - 1.0
    cy = (OD[:,:,:,1]-OD[:,:,:,2]) / (np.sqrt(3.0)*D)

    D = np.expand_dims(D,3)
    cx = np.expand_dims(cx,3)
    cy = np.expand_dims(cy,3)

    X_HSD = np.concatenate((D,cx,cy),3)
    return X_HSD


def clean_thumbnail(thumbnail):
    thumbnail_arr = np.asarray(thumbnail)

    # writable thumbnail
    wthumbnail = np.zeros_like(thumbnail_arr)
    wthumbnail[:, :, :] = thumbnail_arr[:, :, :]

    # Remove pen marking here
    # We are skipping this

    # This  section sets regoins with white spectrum as the backgroud regoin
    thumbnail_std = np.std(wthumbnail, axis=2)
    wthumbnail[thumbnail_std<5] = (np.ones((1,3), dtype="uint8")*255)
    thumbnail_HSD = RGB2HSD( np.array([wthumbnail.astype('float32')/255.]) )[0]
    kernel = np.ones((30,30),np.float32)/900
    thumbnail_HSD_mean = cv2.filter2D(thumbnail_HSD[:,:,2],-1,kernel)
    wthumbnail[thumbnail_HSD_mean<0.05] = (np.ones((1,3),dtype="uint8")*255)
    return wthumbnail


# (Placeholder) Your segmentation function: slide → tissue_mask at level=1
def compute_tissue_mask(slide):
    # Returns a NumPy 2D array of 0/1 at level=1
    # You should replace this with your actual mask logic
    lvl = 1
    thumb = slide.get_thumbnail(slide.level_dimensions[lvl])

    arr = np.array(thumb.convert("L"))  # e.g. simple threshold
    tissue_mask = (arr < 200).astype(np.uint8)

    return tissue_mask




In [ ]:
import os
import numpy as np
import tqdm
import openslide
from google.colab import auth
from google.cloud import storage
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline


# output directory for barcodes
os.makedirs('barcodes-train', exist_ok=True)
os.makedirs('barcodes-test', exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tile_encoder
# transform

# main loop
SPLIT = 'train'

slides = list_slides(SPLIT)

for slide_path in slides[:50]:
  wsi_id = os.path.basename(slide_path).split('.')[0]
  gcs_embedding_path = f'patch-embeddings/{SPLIT}/{wsi_id}_embeddings.pt'

  # if slide is already patched and embeddings are saved - skip
  if client.bucket(bucket.name).blob(gcs_embedding_path).exists():
    print(f'{wsi_id} already processed and embeddings saved in GSC. Skipping.')
    continue

  # processing WSI
  print(f' ----------- Processing WSI {wsi_id} -------------')

  # download and open wsi on colab
  local_svs = f'{wsi_id}.svs'
  client.bucket(bucket.name).blob(slide_path).download_to_filename(local_svs)
  slide = openslide.open_slide(local_svs)

  # compute tissue mask at level 1
  tissue_mask = compute_tissue_mask(slide)

  # patching dimnesions for 40x slides
  objective_power = int(slide.properties['openslide.objective-power']) # 40
  lvl = 1
  downsample = slide.level_downsamples[lvl]
  raw_patch = int((objective_power/20.0)*1000 / downsample)
  patch_size = raw_patch // 2

  # build mask ratios
  w0, h0 = slide.dimensions
  mask_h, mask_w = tissue_mask.shape
  mask_hr = (mask_h / h0) * patch_size
  mask_wr = (mask_w / w0) * patch_size

  # slide over patches
  patches = []
  for i, hi in enumerate(range(0, h0, patch_size)):
        row = []
        for j, wi in enumerate(range(0, w0, patch_size)):
            mi = int(i * mask_hr)
            mj = int(j * mask_wr)
            pm = tissue_mask[mi:mi+int(mask_hr), mj:mj+int(mask_wr)]
            cov = pm.sum()/pm.size
            row.append({'loc':[i,j], 'wsi_loc':[hi,wi], 'tissue_coverage':cov})
        patches.append(row)

  #  Extract RGB histograms for patches ≥70% tissue
  flat = [p for row in patches for p in row]
  hist_feats = []
  sel = []
  for p in flat:
      if p['tissue_coverage'] < 0.7: continue
      hi, wi = p['wsi_loc']
      region = slide.read_region((wi,hi), lvl, (raw_patch, raw_patch)) \
                    .convert("RGB").resize((patch_size, patch_size))
      arr = np.array(region)/255.
      p['rgb_histogram'] = arr.reshape(-1,3).mean(0)
      hist_feats.append(p['rgb_histogram'])
      sel.append(p)
  selected = np.array(hist_feats)


  # 7. KMeans clustering & mosaic selection
  K = 9
  kmeans = KMeans(n_clusters=K).fit(selected)
  for p, lbl in zip(sel, kmeans.labels_): p['cluster_lbl']=int(lbl)

  mosaic = []
  pct = 5
  for c in range(K):
      group = [p for p in sel if p['cluster_lbl']==c]
      n = max(1, int(len(group)*pct/100.))
      km = KMeans(n_clusters=n)
      d = km.fit_transform([p['wsi_loc'] for p in group])
      chosen = set()
      for idx in range(n):
          for s in np.argsort(d[:,idx]):
              if s not in chosen:
                  chosen.add(s)
                  mosaic.append(group[s])
                  break

  # Encode mosaic patches with GigaPath encoder in batches of 20
  embeddings = []
  B = 20
  batch_imgs, batch_idx = [], []
  for mi, p in enumerate(mosaic):
      hi, wi = p['wsi_loc']
      raw = slide.read_region((wi,hi), 0, (int((objective_power/20)*1000),)*2) \
                   .convert("RGB")
      img = transform(raw).unsqueeze(0).to(device)
      batch_imgs.append(img); batch_idx.append(mi)
      if len(batch_imgs)==B:
          inp = torch.cat(batch_imgs,0)
          with torch.no_grad(): out = tile_encoder(inp)
          for b, emb in enumerate(out):
            mosaic[batch_idx[b]]['gigapath_embedding']=emb.cpu().numpy()
          batch_imgs, batch_idx = [], []

  # leftover
  if batch_imgs:
      inp = torch.cat(batch_imgs,0)
      with torch.no_grad(): out = tile_encoder(inp)
      for b, emb in enumerate(out):
          mosaic[batch_idx[b]]['gigapath_embedding']=emb.cpu().numpy()


  # save embeddings to google cloud at embedding path
  emb_list = [p['gigapath_embedding'] for p in mosaic]
  emb_tensor = torch.tensor(emb_list)
  print(emb_tensor.shape)

  local_pt = f"{wsi_id}_embeddings.pt"
  torch.save(emb_tensor, local_pt)

  blob = client.bucket(bucket.name).blob(gcs_embedding_path)
  blob.upload_from_filename(local_pt)
  print(f"Uploaded embeddings to {gcs_embedding_path}")

  os.remove(local_svs)
  torch.cuda.empty_cache()
  del slide, patches, mosaic




BRACS_1003728 already processed and embeddings saved in GSC. Skipping.
BRACS_1379 already processed and embeddings saved in GSC. Skipping.
 ----------- Processing WSI BRACS_1486 -------------
torch.Size([451, 1536])
Uploaded embeddings to patch-embeddings/train/BRACS_1486_embeddings.pt
 ----------- Processing WSI BRACS_1494 -------------
